In [3]:
import bs4, ssl
import pandas as pd
import re
from urllib.request import urlopen, Request
from urllib.error import HTTPError
import time
import datetime, time # for sleep
import platform
from urllib.parse import quote

# SDS Proxy
# import os
# os.environ["HTTP_PROXY"] = "http://70.10.15.10:8080"
# os.environ["HTTPS_PROXY"] = "http://70.10.15.10:8080"
# os.environ["PYTHONHTTPSVERIFY"] = "0"

context = ssl._create_unverified_context()

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                            'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                            'Chrome/23.0.1271.64 Safari/537.11', \
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
            'Accept-Encoding': 'none', \
            'Accept-Language': 'en-US,en;q=0.8', \
            'Connection': 'keep-alive'}

def __Last_Page_Numb_Market(Market) :
    if Market == "KOSPI" :
        url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page=1"
        # print(url)
    elif Market == "KOSDAQ" :
        url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSDAQ&page=1"
        # print(url)

    html = urlopen(url, context=context)
    bs_object = bs4.BeautifulSoup(html, "html.parser")
    main1_pages = bs_object.find_all("a")
    for a_list in main1_pages:
        if a_list is main1_pages[-1] :
            last_page=a_list.attrs['href']
        else :
            continue
        page_num: object=last_page.split('=')
        print("# Market Page_Number : ", page_num[2])
        return page_num[2]

def __Market_Price(company_code, page):
    df_list, date_list, closing_list = [], [], []
    # print("# Market : %s" % company_code)

    print("# Company : %s" % company_code)
    page = int(page)
    # for page_num in range(1, page) :
    for page_num in range(1, 200) :
        url_main = "https://finance.naver.com/sise/sise_index_day.nhn?code=" + company_code + "&page=" + str(page_num)
        # print("# url : %s" % url_main)
        code_data = pd.read_html(url_main, header=0, skiprows=(6,7,8)) # ---> code data로 출력을 생성하면 NaN이 1개라인이 남음, Type은 <class 'list'>
        df_list.append(pd.read_html(url_main)[0]) # ---> List 로 append, 본문[0]과 페이지 출력[1] 부분 중 본문을 출력

    df=pd.concat(df_list)
    df.dropna(subset=["날짜"], inplace=True) # ---> Naver에서 출력되는 가운데 날짜 컬럼 NaN 라인들 삭제처리
    df.rename(columns={'날짜': 'date'}, inplace = True)
    df.rename(columns={'체결가': 'closing'}, inplace = True)
    df.rename(columns={'전일비': 'diff'}, inplace = True)
    df['new_date'] = pd.to_datetime(df['date']) # ---> dateime 형식으로 new_date 추가
    date_list=df.date.values.tolist()
    closing_list=df.closing.values.tolist()
    for x in range(0, len(closing_list)-1):
        closing_list[x] = int(closing_list[x]) # ---> float 에서 int로 변경

    # date_list.reverse()
    # closing_list.reverse()

    return date_list, closing_list

def __DC_Last_Page_Find(page) :

    print("# DCinside Function Start")
    Table_DF = pd.DataFrame()

    for page_num in range(1,page) :
        sleep_count = page_num % 10
        if sleep_count == 0 :
            print("# Sleep Time : 10s")
            print("# Sleep Count : %s" % sleep_count)
            time.sleep(10)

        url = "https://gall.dcinside.com/board/lists/?id=neostock&page=" + str(page_num)
        print(url)

        req = Request(url=url, headers=headers)

        try :
            html = urlopen(req).read()
            # html = urlopen(url, context=context)
        except HTTPError as err:
            if err.code == 404 :
                print("# Error HTTP : 404")
                return
            elif err.code == 403 :
                print("# Error HTTP : 403")
                return
            elif err.code == 500 :
                print("# Error HTTP : 500")
                return
            elif err.code == 505:
                print("# Error HTTP : 505")
                return
            else:
                raise

        bs_object = bs4.BeautifulSoup(html, "html.parser")
        # print(bs_object)
        table1 = bs_object.find_all("table", {'class' : 'gall_list'})
        table2 = bs_object.find_all("td", {'class' : 'gall_date'})
        # print(table)
        # print(test)
        table_data = pd.read_html(str(table1), header=0) # ---> pd.read_html은 List 형태로 보내고 그 안에 DataFrame
        Report_Data = table_data[0] # ---> [1] 일 경우는 하단의 페이지 번호가 출력
        # print(Report_Data)
        ahref_list = []
        for x in table2 :
            ahref = x.get('title')
            ahref_list.append(ahref)
        Report_Data['dates'] = ahref_list # 2021-02-26 21:29:55
        Report_Data = Report_Data.drop(Report_Data[Report_Data['번호']=='AD'].index, axis=0)
        Report_Data = Report_Data.drop(Report_Data[Report_Data['번호']=='공지'].index, axis=0)
        Report_Data['dates'] = pd.to_datetime(Report_Data['dates'], format='%Y-%m-%d %H:%M:%S') # 2021-02-26 21:29:55
        Report_Data['new_dates'] = Report_Data['dates'].dt.strftime('%Y.%m.%d')

        # Table_DF = Table_DF.append(Report_Data['new_dates'])
        Table_DF = Table_DF.append(Report_Data)

        # time.sleep(1) # urllib.error.URLError: <urlopen error [Errno 60] Operation timed out>

    Group_DC = Table_DF.groupby('new_dates').count()
    # print("# Group_Action_Table : ")
    # print(Group_DC['번호'])
    # output = open("DC_index.txt", 'w+t')
    # output.write(Group_DC)
    # Group_DC['번호'].to_csv("DC_index.txt")
    Report_DC = pd.DataFrame(Group_DC['번호'])

    return Report_DC

def __CL_Last_Page_Find(page) :

    print("# Clien Function Start")
    Table_DF = pd.DataFrame()

    for page_num in range(0,page) :
    # for page_num in range(0,2) :
        sleep_count = page_num % 10
        if sleep_count == 0 :
            print("# Sleep Time : 10s")
            print("# Sleep Count : %s" % sleep_count)
            time.sleep(10)

        url = "https://www.clien.net/service/board/cm_stock?&od=T31&po=" + str(page_num) # Stock
        # url = "https://www.clien.net/service/board/cm_vcoin?&od=T31&po=" + str(page_num) # Coin
        print(url)

        req = Request(url=url, headers=headers)

        try :
            html = urlopen(req).read()
            # html = urlopen(url, context=context)
        except HTTPError as err:
            if err.code == 404 :
                print("# Error HTTP : 404")
                return
            elif err.code == 403 :
                print("# Error HTTP : 403")
                return
            elif err.code == 500 :
                print("# Error HTTP : 500")
                return
            elif err.code == 505:
                print("# Error HTTP : 505")
                return
            else:
                raise

        bs_object = bs4.BeautifulSoup(html, "html.parser")
        # print(bs_object)
        table = bs_object.find_all("span", {'class' : 'timestamp'})
        # print(type(table))
        # print(table)
        date_list = []
        for x in table :
            # print(type(x.text))
            if x.text != '2014-10-24 15:46:09' and x.text != '2020-03-27 15:14:21' \
                and x.text != '2021-02-26 17:12:17' and x.text != '2021-01-21 12:46:02' and x.text != '2020-07-22 21:37:38' :
                # print(x.text)
                date_list.append(x.text)

        Report_Data = pd.DataFrame({'dates' : date_list})
        Report_Data['dates'] = pd.to_datetime(Report_Data['dates'], format='%Y-%m-%d %H:%M:%S') # 2021-02-26 21:29:55
        Report_Data['new_dates'] = Report_Data['dates'].dt.strftime('%Y.%m.%d')
        # print(Report_Data)
        # Report_Data['Count'] = '1'
        Group_CL = Report_Data.groupby('new_dates').count()
        # print(Group_CL)

        # Report_Data['dates'] = pd.to_datetime(Report_Data['dates'], format='%Y-%m-%d %H:%M:%S') # 2021-02-26 21:29:55
        # Report_Data['new_dates'] = Report_Data['dates'].dt.strftime('%Y.%m.%d')

        Table_DF = Table_DF.append(Group_CL)

        time.sleep(2) # urllib.error.URLError: <urlopen error [Errno 60] Operation timed out>

    Table_DF = Table_DF.groupby('new_dates').sum()

    return Table_DF

def __New_Make_Chart (Market_1, Market_2, Date_List, Market_List, Count_List) :
    # __New_Make_Chart ("KOSPI", "DC_Inside", Date_List, Market_List, Count_List)

    DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")

    print("# Platform : %s" % platform.system())

    if platform.system() == 'Darwin' :
        File_Name = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/"+Market_1+"_"+Market_2+"_"+DateTime+".html"
        input_1 = open('/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/New_Echart_Test_1.html', 'r') # Echrt의 머릿글
    else :
        File_Name = "D:\\Python\\Log\\"+Market_1+"_"+Market_2+"_"+DateTime+".html"
        input_1 = open('D:\\Python\\Log\\New_Echart_Test_1.html', 'r') # Echrt의 머릿글

    closing_list_A = Market_List
    closing_list_B = Count_List
    date_list = Date_List

    n = 300
    print("date list 0 : %s" % date_list[0])
    print("date leng : %s" % len(date_list))
    del date_list[n:]
    del Market_List[n:]
    del Count_List[n:]
    print("# After DEL")
    print("date list 0 : %s" % date_list[0])
    print("date leng : %s" % len(date_list))

    output = open(File_Name, 'w+t')
    output.write('<head><meta charset="utf-8"></head>')

    # for idx, val in enumerate(closing_list_A) :
    #     if type(closing_list_A[idx]) == int or type(closing_list_A[idx]) == float :
    #         continue
    #     else :
    #         closing_list_A[idx] = val

    numrow_A = [float(x) for x in closing_list_A]
    max_value_A = int(max(numrow_A)) * 1.5
    numrow_B = [float(x) for x in closing_list_B]
    max_value_B = int(max(numrow_B)) * 1.5 # ---> ValueError: could not convert string to float: '11,550'

    print("# MAKE HTML Start")
    for line in input_1.readlines():
        output.write(line)
    output.write("        data: ['%s', '%s']\n" % (Market_1, Market_2))
    output.write("    },\n")
    output.write("    xAxis: [\n")
    output.write("        {\n")
    output.write("            type: 'category',\n")
    output.write("            axisTick: {\n")
    output.write("                alignWithLabel: true\n")
    output.write("            },\n")
    output.write("            data: [")
    # for item in date_list[::-1] :
    for item in date_list :
        output.write("'%s'," % item)
    output.write("        ]\n")
    output.write("        }\n")
    output.write("    ],\n")
    output.write("    yAxis: [\n")
    output.write("        {\n")
    output.write("            type: 'value',\n")
    output.write("            name: '%s',\n" % Market_1)
    output.write("            min: 0,\n")
    output.write("            max: %s,\n" % str(max_value_A))
    output.write("            position: 'right',\n")
    output.write("            axisLine: {\n")
    output.write("                lineStyle: {\n")
    output.write("                    color: colors[0]\n")
    output.write("                }\n")
    output.write("            },\n")
    output.write("            axisLabel: {\n")
    output.write("                formatter: '{value} pt'\n")
    output.write("            }\n")
    output.write("        },\n")
    output.write("        {\n")
    output.write("            type: 'value',\n")
    output.write("            name: '%s',\n" % Market_2)
    output.write("            min: 0,\n")
    output.write("            max: %s,\n" % str(max_value_B))
    output.write("            position: 'left',\n")
    output.write("            axisLine: {\n")
    output.write("                lineStyle: {\n")
    output.write("                    color: colors[1]\n")
    output.write("                }\n")
    output.write("            },\n")
    output.write("            axisLabel: {\n")
    output.write("                formatter: '{value} pt'\n")
    output.write("            }\n")
    output.write("        }\n")
    output.write("    ],\n")
    output.write("    series: [\n")
    output.write("        {\n")
    output.write("            name: '%s',\n" % Market_1)
    output.write("            type: 'line',\n")
    output.write("              data: [\n")
    # for x in closing_list_A[::-1] :
    for x in closing_list_A :
        output.write("'%s'," % x)
    output.write("        ]\n")
    output.write("        },\n")
    output.write("        {\n")
    output.write("            name: '%s',\n" % Market_2)
    output.write("            type: 'line',\n")
    output.write("            yAxisIndex: 1,\n")
    output.write("              data: [\n")
    # for x in closing_list_B[::-1]  :
    for x in closing_list_B :
        output.write("'%s'," % x)
    output.write("    ]\n")
    output.write("        }\n")
    output.write("    ]\n")
    output.write("};\n")
    output.write("        // use configuration item and data specified to show chart\n")
    output.write("        myChart.setOption(option);\n")
    output.write("    </script>\n")
    output.write("</body>\n")

    # for line in input_2.readlines():
        # output.write(line)

    print("# MAKE HTML Done")

    output.close()

if(__name__ == '__main__') :
    print("# Program Start")

    Main_DF = pd.DataFrame()

    KOSPI_Page = __Last_Page_Numb_Market("KOSPI") # ---> 최근 1년이면 굳이 모두 할 필요없다.
    KOSPI_Date, KOSPI_List = __Market_Price("KOSPI", KOSPI_Page)
    KOSPI = pd.DataFrame({'new_dates' : KOSPI_Date, 'kospi': KOSPI_List})
    print(KOSPI)
    KOSPI.to_csv("KOSPI.txt")

    KOSDAQ_Page = __Last_Page_Numb_Market("KOSDAQ") # ---> 최근 1년이면 굳이 모두 할 필요없다.
    KOSDAQ_Date, KOSDAQ_List = __Market_Price("KOSDAQ", KOSDAQ_Page)
    KOSDAQ = pd.DataFrame({'date' : KOSDAQ_Date, 'kosdaq': KOSDAQ_List})
    print(KOSDAQ)
    KOSDAQ.to_csv("KOSDAQ.txt")

    DC_Check = 1
    CL_Check = 1

    if CL_Check == 1 :
        DateTime = datetime.datetime.today().strftime("%Y%m%d")
        # CL_page = 1335 # 2021.05.21
        CL_page = 500
        # Coin_page = 2716

        CL_Action_Table = __CL_Last_Page_Find(CL_page)
        # CL_Action_Table = __CL_Last_Page_Find(Coin_page)
        print("# CL")
        print(CL_Action_Table)
        CL_Action_Table.to_csv("CL_Action_Table"+DateTime+".txt")

        ############################################################
        # Clien & KOSPI
        KOSPI_CL = CL_Action_Table.join(KOSPI.set_index('new_dates'), on='new_dates')
        KOSPI_CL.dropna(subset=['kospi'], inplace=True)
        print("# KOSPI_CL")
        print(KOSPI_CL)
        KOSPI_CL = KOSPI_CL.reset_index()
        print("# KOSPI_CL 3")
        print(KOSPI_CL)
        Date_List = KOSPI_CL['new_dates'].to_list()
        Market_List = KOSPI_CL['kospi'].to_list()
        Count_List = KOSPI_CL['dates'].to_list()
        __New_Make_Chart ("KOSPI", "Clien", Date_List, Market_List, Count_List)

        ############################################################
        # Clien & KOSDAQ
        print("# KOSDAQ")
        print(KOSDAQ)
        KOSDAQ.rename(columns={'date': 'new_dates'}, inplace = True)
        KOSDAQ_CL = CL_Action_Table.join(KOSDAQ.set_index('new_dates'), on='new_dates')
        KOSDAQ_CL.dropna(subset=['kosdaq'], inplace=True)
        print("# KOSDAQ_CL")
        print(KOSDAQ_CL)
        KOSDAQ_CL = KOSDAQ_CL.reset_index()
        print("# KOSDAQ_CL 3")
        print(KOSDAQ_CL)
        Date_List = KOSDAQ_CL['new_dates'].to_list()
        Market_List = KOSDAQ_CL['kosdaq'].to_list()
        Count_List = KOSDAQ_CL['dates'].to_list()
        __New_Make_Chart ("KOSDAQ", "Clien", Date_List, Market_List, Count_List)

    if DC_Check == 1 :
        DateTime = datetime.datetime.today().strftime("%Y%m%d")
        # DC_page = 11779
        # DC_page = 10000
        DC_page = 500

        DC_Action_Table = __DC_Last_Page_Find(DC_page)
        print("# DC")
        print(DC_Action_Table)
        DC_Action_Table.to_csv("DC_Action_Table"+DateTime+".txt")

        ############################################################
        # DC & KOSPI
        KOSPI_DC = DC_Action_Table.join(KOSPI.set_index('new_dates'), on='new_dates')
        print("# KOSPI_DC 1")
        print(KOSPI_DC)
        KOSPI_DC.dropna(subset=['kospi'], inplace=True) 
        print("# KOSPI_DC 2")
        print(KOSPI_DC)
        KOSPI_DC = KOSPI_DC.reset_index()
        print("# KOSPI_DC 3")
        print(KOSPI_DC)
        Date_List = KOSPI_DC['new_dates'].to_list()
        Market_List = KOSPI_DC['kospi'].to_list()
        Count_List = KOSPI_DC['번호'].to_list()

        __New_Make_Chart ("KOSPI", "DC_Inside", Date_List, Market_List, Count_List)

        ###########################################################
        # DC & KOSDAQ
        KOSDAQ_DC = DC_Action_Table.join(KOSDAQ.set_index('new_dates'), on='new_dates')
        print("# KOSDAQ_DC 1")
        print(KOSDAQ_DC)
        KOSDAQ_DC.dropna(subset=['kosdaq'], inplace=True) 
        print("# KOSDAQ_DC 2")
        print(KOSDAQ_DC)
        KOSDAQ_DC = KOSDAQ_DC.reset_index()
        print("# KOSDAQ_DC 3")
        print(KOSDAQ_DC)
        Date_List = KOSDAQ_DC['new_dates'].to_list()
        Market_List = KOSDAQ_DC['kosdaq'].to_list()
        Count_List = KOSDAQ_DC['번호'].to_list()

        __New_Make_Chart ("KOSDAQ", "DC_Inside", Date_List, Market_List, Count_List)

    print("# Value_Find for STATEMENTS End")

# Program Start
# Market Page_Number :  1366
# Company : KOSPI
       new_dates    kospi
0     2021.06.29  3286.00
1     2021.06.28  3301.00
2     2021.06.25  3302.00
3     2021.06.24  3286.00
4     2021.06.23  3276.00
...          ...      ...
1189  2016.08.25  2042.00
1190  2016.08.24  2043.00
1191  2016.08.23  2049.00
1192  2016.08.22  2042.00
1193  2016.08.19  2056.24

[1194 rows x 2 columns]
# Market Page_Number :  1047
# Company : KOSDAQ
            date  kosdaq
0     2021.06.29  1022.0
1     2021.06.28  1017.0
2     2021.06.25  1012.0
3     2021.06.24  1012.0
4     2021.06.23  1016.0
...          ...     ...
1189  2016.08.25   683.0
1190  2016.08.24   687.0
1191  2016.08.23   687.0
1192  2016.08.22   688.0
1193  2016.08.19   696.2

[1194 rows x 2 columns]
# Clien Function Start
# Sleep Time : 10s
# Sleep Count : 0
https://www.clien.net/service/board/cm_stock?&od=T31&po=0
https://www.clien.net/service/board/cm_stock?&od=T31&po=1
https://www.clien.net/service/board/cm_stock?&od=T3

https://gall.dcinside.com/board/lists/?id=neostock&page=2
https://gall.dcinside.com/board/lists/?id=neostock&page=3
https://gall.dcinside.com/board/lists/?id=neostock&page=4
https://gall.dcinside.com/board/lists/?id=neostock&page=5
https://gall.dcinside.com/board/lists/?id=neostock&page=6
https://gall.dcinside.com/board/lists/?id=neostock&page=7
https://gall.dcinside.com/board/lists/?id=neostock&page=8
https://gall.dcinside.com/board/lists/?id=neostock&page=9
# Sleep Time : 10s
# Sleep Count : 0
https://gall.dcinside.com/board/lists/?id=neostock&page=10
https://gall.dcinside.com/board/lists/?id=neostock&page=11
https://gall.dcinside.com/board/lists/?id=neostock&page=12
https://gall.dcinside.com/board/lists/?id=neostock&page=13
https://gall.dcinside.com/board/lists/?id=neostock&page=14
https://gall.dcinside.com/board/lists/?id=neostock&page=15
https://gall.dcinside.com/board/lists/?id=neostock&page=16
https://gall.dcinside.com/board/lists/?id=neostock&page=17
https://gall.dcinside.com/b